<a href="https://colab.research.google.com/github/oszac/M1.-Actividad/blob/main/M1_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M1. Actividad

## Imports

- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

In [ ]:
!pip install mesa

     |████████████████████████████████| 691 kB 4.3 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 


In [ ]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid
from mesa.space import MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random

## Crear el modelo

In [ ]:
class Celda(Agent):
  """
  Representa la celda que pondremos en nuestro Modelo
  su estado en 1 significa que esta sucia y 0 cuando está limpia
  """

  def __init__(self, unique_id, model, state):
    super().__init__(unique_id, model)
    self.pos = unique_id
    self.next_state = None
    self.state = state # Para saber si esta limpia o sucia 

In [ ]:
class Aspiradora(Agent):
  """
  Representa a un agente aspiradora
  """
  def __init__(self, unique_id, model):
    """
    Crea las aspiradoras
    """
    super().__init__(unique_id, model)
    #self.live = np.random.choice([0,1])
    self.next_pos = None
    self.movimientos = 0
    
  def step(self):

    neighbours = self.model.grid.get_neighbors(
      self.pos,
      moore=True,
      include_center=True)

    for neighbour in neighbours: 
      if isinstance(neighbour, Celda) and self.pos == neighbour.pos:
        if neighbour.state == 1: 
          neighbour.next_state = 0
          self.next_pos = self.pos
        else : 
          vecinos_alrededor = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
          neighbour.next_state = 0
          self.next_pos = self.random.choice(vecinos_alrededor)
        break

  def advance(self):
    """
    Define el nuevo estado de la aspiradora
    """
    neighbours = self.model.grid.get_neighbors(
                self.pos,
                moore=True,
                include_center=True)

    for neighbor in neighbours:
      if isinstance(neighbor, Celda) and self.pos == neighbor.pos:
        neighbor.state = neighbor.next_state
        break

    self.model.grid.move_agent(self, self.next_pos)
    self.movimientos += 1

  def movimientos_totales(self):
    return self.movimientos


            
class Habitacion(Model):
  """
  Define el modelo del juego de la vida.
  """
  def __init__(self, width, height, porcentaje_celdas_sucias, num_agentes):
    self.num_agentes = num_agentes
    self.tamaño = width * height
    self.porcentaje_celdas_sucias = porcentaje_celdas_sucias
    self.porcentaje_celdas_limpias = 1 - porcentaje_celdas_sucias
    self.grid = MultiGrid(width, height, False)
    self.schedule = SimultaneousActivation(self)
    self.listaAgentes = []
      
    # Colocamos las celdas en el grid
    # total_celdas_sucias = int(width * height * porcentaje_celdas_sucias)
    global TOTAL_CELDAS_SUCIAS
    for celda in self.grid.coord_iter():
      contenido_celda, x, y = celda
      aleatorio = random.randint(1, 2)
      if aleatorio == 1 and TOTAL_CELDAS_SUCIAS > 0:
        a = Celda((x, y), self, 1) # Celda sucia asignada con valor de 1
        TOTAL_CELDAS_SUCIAS -= 1
      else:
        a = Celda((x, y), self, 0) # Celda limpia asignada con valor de 0
      self.grid.place_agent(a, (x, y))
      self.schedule.add(a)

    # Colocamos las aspiradoras
    for identificador in range(num_agentes):
      a = Aspiradora(identificador, self)
      self.grid.place_agent(a, (1, 1))
      self.schedule.add(a)
      self.listaAgentes.append(a)

    # Aquí definimos el colector de datos para obtener el grid completo.
    self.datacollector = DataCollector(model_reporters={"Grid": self.get_grid})
    
  def step(self):
    """
    En cada paso el colector toma la información que se definió y almacena el grid para luego
    graficarlo.
    """
    self.datacollector.collect(self)
    self.schedule.step()

  def todo_limpio(self):
    for celda in self.grid.coord_iter():
      contenido_celda, x, y = celda
      for objeto in contenido_celda:
        if isinstance(objeto, Celda) and objeto.state == 1:
          return False
    global PORCENTAJE_LIMPIAS
    PORCENTAJE_LIMPIAS = (TOTAL_CELDAS_SUCIAS / (self.tamaño) ) * 100
    return True

  def get_grid(self):
    """
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    :param model: El modelo del cual obtener el grid.
    :return: Matriz con la información del grid del agente.
    """

    # Generamos la grid para contener los valores
    grid = np.zeros((self.grid.width, self.grid.height))

    # Asignamos una celda a cada elemento
    for cell in self.grid.coord_iter():
        cell_content, x, y = cell
        for objeto in cell_content:
          if isinstance(objeto, Aspiradora):
            grid[x][y] = 2
          elif isinstance(objeto, Celda):
            grid[x][y] = objeto.state

    return grid

  def todos_agentes(self):
    return self.listaAgentes
  
  #def porcentaje_limpias(self):
   # return self.total_celdas_sucias


# Ejecución del modelo
A continuación corremos el modelo

In [ ]:
# Definimos del tamaño de la habitacion
M = 10
N = 10

# Definimos el número de agentes
NUM_AGENTES = 5

# Porcentaje de celdas sucias
PORCENTAJE_CELDAS_SUCIAS = 0.4

# Cantidad de celdas sucias
TOTAL_CELDAS_SUCIAS = int(N * M * PORCENTAJE_CELDAS_SUCIAS)
PORCENTAJE_LIMPIAS = 0.0

# Tiempo maximo de ejecucion
TIEMPO_MAXIMO = 1

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()

model = Habitacion(M, N, PORCENTAJE_CELDAS_SUCIAS, NUM_AGENTES)

while((time.time() - start_time) < TIEMPO_MAXIMO) and not model.todo_limpio():
  model.step()

tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))
agentes_modelo = model.todos_agentes()

movimientos_totales = 0
for agente in agentes_modelo:
  movimientos_totales += agente.movimientos_totales()


# Informe


In [ ]:
print("Tiempo necesario hasta que todas las celdas estan limpias: ", tiempo_ejecucion, "/", start_time)
print("Número de movimientos realizados por los agentes:", movimientos_totales)
print("Porcentaje de celdas limpias al finalizar la ejecucion:", PORCENTAJE_LIMPIAS)

Tiempo necesario hasta que todas las celdas estan limpias:  0:00:00.161017 / 1653261536.9204636
Número de movimientos realizados por los agentes: 1555
Porcentaje de celdas limpias al finalizar la ejecucion: 0.0


Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información.

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

Graficamos la información usando `matplotlib`

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
anim